In [11]:
# Created new env - since mysql connector did not work with Python 3.8.0. New env has python 3.7

In [1]:
import pandas as pd
import numpy as np
import pymssql
import mysql.connector

In [9]:
pd.set_option('display.width', 75)
pd.set_option('display.max_columns', 5)
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
import watermark
%load_ext watermark

In [3]:
%watermark -n -v -g -iv

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.20.0

Git hash: cc72f8009bccf918348e746b7bc9945d84bf36e0

watermark: 2.1.0
numpy    : 1.19.2
pandas   : 1.2.1
pymssql  : 2.1.5
json     : 2.0.9



In [5]:
query = 'SELECT studentid, school, sex, age, famsize, \
medu AS mothereducation, fedu AS fathereducation, traveltime, \
studytime, failures, famrel, freetime,  goout, g1 AS gradeperiod1, \
g2 AS gradeperiod2, g3 AS gradeperiod3 From studentmath'

In [6]:
server = 'pdcc.c9sqqzd5fulv.us-west-2.rds.amazonaws.com'

user = 'pdccuser'
password = 'pdccpass'
database = 'pdcctest'

conn = pymssql.connect(server=server, user=user, password=password, database=database)
studentmath = pd.read_sql(query,conn)
conn.close()

In [7]:
studentmath.dtypes

studentid          object
school             object
sex                object
age                 int64
famsize            object
mothereducation     int64
fathereducation     int64
traveltime          int64
studytime           int64
failures            int64
famrel              int64
freetime            int64
goout               int64
gradeperiod1        int64
gradeperiod2        int64
gradeperiod3        int64
dtype: object

In [10]:
studentmath.head()

,studentid,school,...,gradeperiod2,gradeperiod3
0,001,GP,...,6,6
1,002,GP,...,5,6
2,003,GP,...,8,10
3,004,GP,...,14,15
4,005,GP,...,10,10


In [12]:
# Alternative - Use the mysql connector and read_sql to get data from MySQL

In [13]:
host = 'pdccmysql.c9sqqzd5fulv.us-west-2.rds.amazonaws.com'

user = 'pdccuser'
password = 'pdccpass'
database = 'pdccschema'

In [15]:
connmysql = mysql.connector.connect(host=host, database=database, user=user, password=password)
studentmath = pd.read_sql(query,connmysql)
connmysql.close()

In [16]:
studentmath.head()

,studentid,school,...,gradeperiod2,gradeperiod3
0,1,GP,...,6,6
1,2,GP,...,5,6
2,3,GP,...,8,10
3,4,GP,...,14,15
4,5,GP,...,10,10


In [17]:
studentmath.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   studentid        395 non-null    int64 
 1   school           395 non-null    object
 2   sex              395 non-null    object
 3   age              395 non-null    int64 
 4   famsize          395 non-null    object
 5   mothereducation  395 non-null    int64 
 6   fathereducation  395 non-null    int64 
 7   traveltime       395 non-null    int64 
 8   studytime        395 non-null    int64 
 9   failures         395 non-null    int64 
 10  famrel           395 non-null    int64 
 11  freetime         395 non-null    int64 
 12  goout            395 non-null    int64 
 13  gradeperiod1     395 non-null    int64 
 14  gradeperiod2     395 non-null    int64 
 15  gradeperiod3     395 non-null    int64 
dtypes: int64(13), object(3)
memory usage: 49.5+ KB


In [18]:
# rearrange columns and set an index
newcolorder = ['studentid', 'gradeperiod1', 'gradeperiod2',
  'gradeperiod3', 'school', 'sex', 'age', 'famsize',
  'mothereducation', 'fathereducation', 'traveltime',
  'studytime', 'freetime', 'failures', 'famrel',
  'goout']

In [19]:
studentmath = studentmath[newcolorder]

In [20]:
studentmath.studentid.count()

395

In [21]:
studentmath.studentid.nunique()

395

In [22]:
studentmath.set_index('studentid', inplace=True)

In [23]:
studentmath.count()

gradeperiod1       395
gradeperiod2       395
gradeperiod3       395
school             395
sex                395
age                395
famsize            395
mothereducation    395
fathereducation    395
traveltime         395
studytime          395
freetime           395
failures           395
famrel             395
goout              395
dtype: int64

In [24]:
# add codes to data values
setvalues={'famrel':{1:'1:very bad',2:'2:bad',3:'3:neutral',
    4:'4:good',5:'5:excellent'},
  'freetime':{1:'1:very low',2:'2:low',3:'3:neutral',
    4:'4:high',5:'5:very high'},
  'goout':{1:'1:very low',2:'2:low',3:'3:neutral',
    4:'4:high',5:'5:very high'},
  'mothereducation':{0:np.nan,1:'1:k-4',2:'2:5-9',
    3:'3:secondary ed',4:'4:higher ed'},
  'fathereducation':{0:np.nan,1:'1:k-4',2:'2:5-9',
    3:'3:secondary ed',4:'4:higher ed'}}

In [25]:
studentmath.replace(setvalues, inplace=True)
setvalueskeys = [k for k in setvalues]

In [28]:
studentmath.tail()

,gradeperiod1,gradeperiod2,...,famrel,goout
studentid,,,,,
391,9,9,...,5:excellent,4:high
392,14,16,...,2:bad,5:very high
393,10,8,...,5:excellent,3:neutral
394,11,12,...,4:good,1:very low
395,8,9,...,3:neutral,3:neutral


In [26]:
setvalueskeys

['famrel', 'freetime', 'goout', 'mothereducation', 'fathereducation']

In [27]:
studentmath[setvalueskeys].memory_usage(index=False)

famrel             3160
freetime           3160
goout              3160
mothereducation    3160
fathereducation    3160
dtype: int64

In [29]:
for col in studentmath[setvalueskeys].columns:
    studentmath[col] = studentmath[col].astype('category') 

In [30]:
studentmath[setvalueskeys].memory_usage(index=False)

famrel             607
freetime           607
goout              607
mothereducation    599
fathereducation    599
dtype: int64

In [31]:
# Calculate percentages for values in the famrel column.

studentmath['famrel'].value_counts(sort=False,normalize=True)

1:very bad    0.02
2:bad         0.05
3:neutral     0.17
4:good        0.49
5:excellent   0.27
Name: famrel, dtype: float64

In [32]:
# Use apply to calculate percentages for multiple columns:

studentmath[['freetime', 'goout']].apply(pd.Series.value_counts, sort=False, normalize=True)

,freetime,goout
1:very low,0.05,0.06
2:low,0.16,0.26
3:neutral,0.40,0.33
4:high,0.29,0.22
5:very high,0.10,0.13


In [33]:
studentmath[['mothereducation', 'fathereducation']].apply(pd.Series.value_counts, sort=False, normalize=True)

,mothereducation,fathereducation
1:k-4,0.15,0.21
2:5-9,0.26,0.29
3:secondary ed,0.25,0.25
4:higher ed,0.33,0.24
